# This generates a file containing the MJ background estimation from the ABCD method:
## A/B = C/D 
Where,
A = Signal Region (SR)
B = Signal Region Same Sign (SR-SS)
C = Control Region (CR)
D = Control Region Same Sign (CR-SS)

$$\frac{\text{SR}}{\text{SR}_{\text{SS}}} = \frac{\text{CR}}{\text{CR}_{\text{SS}}}$$


$$ \text{SR} = \text{SR}_{\text{SS}} \text{RQCD} $$

### Signal region path$
### Multi jet control region path
### Sample used for Z-QCDjets

In [1]:
import ROOT as r
from ROOT import gStyle
import numpy as np
import ctypes
import os
from IPython.display import display 
import pandas as pd 
from histogramHelpers import *


# Define the path to the channel and the directories for the regions.
channelPath = "/Users/diegomac/Documents/HEP/VBF-Analysis/TauTau/TauhadTaulep/High-Mass/"
SR = "DeterministicBDTRecVisCutSR/"
SRSS = "DeterministicBDTRecVisCutSRSS/"
CR = "NonIsoOS/"
CRSS = "NonIsoSS/"

# Select EWjj and QCDjj samples
EWjj = "Signal_Sherpa"
QCDjj = "Ztautau_SherpaRW"

RQCDwithCRs = False

# Remove previous MJ.root file
samples=[i for i in os.listdir(channelPath+SR) if ('.root' in i and 'Wjets' not in i)]
print(samples)
if "MJ.root" in samples:
    os.system("rm "+channelPath+SR+"MJ.root")

Welcome to JupyROOT 6.28/04
['MJ.root', 'Ztautau_SherpaRW.root', 'Ztautau_MGRW.root', 'Zjets.root', 'Signal_Sherpa.root', 'W_EWK_PoPy.root', 'Signal_PoPy.root', 'Data.root', 'MC.root', 'VV.root', 'Higgs.root', 'VV_EWK.root', 'W_EWK_Sherpa.root', 'ttbar.root', 'singletop.root']


In [2]:
import sys
sys.path.append('/Users/diegomac/Documents/HEP/VBF-Analysis/Scripts/')
from histogramHelpers import tautauHighMassHistograms
histos = tautauHighMassHistograms

In [3]:
mcSamples = [EWjj,QCDjj]
backgroundSamples = ['VV',"ttbar",'singletop']#+['Wjets',]
if "Tau" in channelPath:
    backgroundSamples += ['Higgs','Zjets','W_EWK_Sherpa','VV_EWK']
mcSamples += backgroundSamples

# Open target file
multiJetFile =r.TFile.Open(channelPath+SR+"MJ.root", "RECREATE")

# First evaluate if there is evidence for MJ BG.
totalMJisZero = False
nGapJetsHistogram = dataSubtract("n_jets_interval",channelPath+SRSS,"Data",mcSamples,histos,rebin=False)
totalMJ = nGapJetsHistogram.GetBinContent(1)
totalMJUncertainty = nGapJetsHistogram.GetBinError(1)
if (totalMJ < 0.0) or (totalMJ/totalMJUncertainty < 1.0 ):
    totalMJisZero = True
    print("NO MJ EVIDENCE!")
print("MJ = ",totalMJ," +- ",totalMJUncertainty)

if totalMJ < 0.0:
    totalMJ = 0.0
    

NO MJ EVIDENCE!
MJ =  -0.026165008544921875  +-  1.7986946311830467


In [4]:
from histogramHelpers import cutsZtautau
for hist in histos:
    # Determine if histogram needs to be rebined
    rebinHistogram = hist.needsRebin()
    
    print(hist.m_name,rebinHistogram)
    
    # Calculate RQCD in MJCRs
    if RQCDwithCRs:
        # Calculate RQCD
        dataSubtractedHistoCR = dataSubtract(hist.m_name,channelPath+CR,"Data",mcSamples,hist,rebin=rebinHistogram)
        dataSubtractedHistoCRSS = dataSubtract(hist.m_name,channelPath+CRSS,"Data",mcSamples,hist,rebin=rebinHistogram)
        makeNegativeBinsZero(dataSubtractedHistoCR)
        makeNegativeBinsZero(dataSubtractedHistoCRSS)
        RQCD = dataSubtractedHistoCR.Integral(1,-1)/dataSubtractedHistoCRSS.Integral(1,-1)
    # If not, use input value
    else:
        RQCD = 1.2
        UncerRQCD = 1.2*0.25
    
    # Calculate the MJ Background shape
    dataSubtractedHistoSRSS = dataSubtract(hist.m_name,channelPath+SRSS,"Data",mcSamples,hist,rebin=rebinHistogram)
    dataSubtractedHistoSRSS.Scale(RQCD)
    makeNegativeBinsZero(dataSubtractedHistoSRSS)
    MJ = dataSubtractedHistoSRSS.Clone()
    
    # If no MJ Evidence fix the bins that are in the final selection
    if totalMJisZero:
        SRhisto = mcAdd(hist.m_name,channelPath+SR,mcSamples,hist,rebin=rebinHistogram)
        cutLeft = hist.m_leftCut
        cutRight = hist.m_rightCut
        makeSRBinsConsistentWithNOMJ(MJ,cutLeft,cutRight,totalMJ,totalMJUncertainty,SRhisto)
        
    # Take into account the uncertainty in RQCD
    scaleUncertainty(MJ,UncerRQCD/RQCD)
    
    # Save histogram in MJ.root file
    multiJetFile.WriteObject(MJ,hist.m_name)
    
    
multiJetFile.Close()

n_bjets False
Integral in SR bins: 10.965399742126465
lepiso False
Integral in SR bins: 10.965399742126465
n_jets_interval False
Integral in SR bins: 10.965399742126465
flavourJet1_basic_all False
Integral in SR bins: 10.965399088454433
flavourJet2_basic_all False
Integral in SR bins: 10.96159770275699
elecPdgID_basic_all False
Integral in SR bins: 4.74889562278986
muonPdgID_basic_all False
Integral in SR bins: 6.198298705741763
tauPdgID_basic_all False
Integral in SR bins: 10.965398792177439
nLightJets_basic_all False
Integral in SR bins: 10.965399384498596
tau_pt True
Integral in SR bins: 10.957665085792542
lep_pt True
Integral in SR bins: 10.965399622917175
delta_phi True
Integral in SR bins: 10.965399265289307
delta_y True
Integral in SR bins: 10.965399265289307
omega True
Integral in SR bins: 10.965398848056793
rnn_score_1p True
Integral in SR bins: 9.146996855735779
rnn_score_3p True
Integral in SR bins: 1.818402886390686
ljet0_pt True
Integral in SR bins: 10.946888163685799
ljet

/Users/diegomac/Documents/HEP/VBF-Analysis/Scripts/histogramHelpers.py:187: RuntimeWarning: invalid value encountered in sqrt
  binError = totalMJUncertainty*np.sqrt(sf)


Integral in SR bins: 10.965399771928787
tauTransMass_basic_all True
Integral in SR bins: 10.916394708212465
signedCentrality_basic_all True
Integral in SR bins: 10.965399343520403
visibleMass_basic_all True
Integral in SR bins: 10.96539906412363
recoVisibleMassRatio True
Integral in SR bins: 10.965399503707886


Warning in <TH1F::Rebin>: overflow entries will not be used when rebinning
Warning in <TH1F::Rebin>: overflow entries will not be used when rebinning
Warning in <TH1F::Rebin>: Bin edge 2 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 3 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 5 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 6 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 2 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bin edge 3 of rebinned histogram does not match any bin edges of the old histogram. Result can be inconsistent
Warning in <TH1F::Rebin>: Bi

In [22]:
from ctypes import *
osE = c_double(0.0)
ssE = c_double(0.0)
osHisto = dataSubtract("mass_jj","/Users/diegomac/Documents/HEP/VBF-Analysis/TauTau/TauhadTaulep/Z-Peak/MJCROS/","Data",mcSamples,histos,rebin=True)
ssHisto = dataSubtract("mass_jj","/Users/diegomac/Documents/HEP/VBF-Analysis/TauTau/TauhadTaulep/Z-Peak/MJCRSS/","Data",mcSamples,histos,rebin=True)
osIntegral = osHisto.IntegralAndError(1,-1,osE)
ssIntegral = ssHisto.IntegralAndError(1,-1,ssE)

print("OS: ",osIntegral)
print("SS: ",ssIntegral)

osE = osE.value
ssE = ssE.value

print("OS E: ",osE)
print("SS E: ",ssE)
print(osIntegral/ssIntegral," +- ",osIntegral/ssIntegral*np.sqrt((osE/osIntegral)**2+(ssE/ssIntegral)))

OS:  94.53706323355436
SS:  88.52085138997063
OS E:  19.725730909960834
SS E:  10.652698160888127
1.0679637819690624  +-  0.43233208949062885
